In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from Glocal_IB import Glocal_IB

In [10]:
# 1. 定义超参数
BATCH_SIZE = 4
SEQ_LEN = 24
FEATURES = 10
EMBEDDING_DIM = 64

In [11]:
class DummyImputationModel(nn.Module):
    """一个简单的伪插补模型"""
    def __init__(self, seq_len, features, embedding_dim):
        super().__init__()
        self.seq_len = seq_len
        self.features = features
        self.embedding_dim = embedding_dim
        self.encoder = nn.Linear(features, embedding_dim)
        self.decoder = nn.Linear(embedding_dim, features)
        print(f"基础模型初始化完毕，输入特征: {features}, 嵌入维度: {embedding_dim}")

    def forward(self, x, **kwargs):
        # 输入 x 维度: (batch, seq_len, features)
        embedding = self.encoder(x)
        
        # 解码器将嵌入向量扩展回序列长度
        # (batch, seq_len, embedding_dim)
        reconstructed = embedding
        output = self.decoder(reconstructed) # -> (batch, seq_len, features)

        # 按照约定，返回插补结果和中间嵌入
        return output, embedding
    
    def get_embedding_dim(self): # 只是为了测试 __getattr__ 功能
        return self.embedding_dim


In [12]:
# 2. 实例化基础模型和 Glocal_IB 包装器
base_model = DummyImputationModel(SEQ_LEN, FEATURES, EMBEDDING_DIM)

# 使用 "cos_align" 作为对齐损失，权重为 0.5
glocal_model = Glocal_IB(
    base_model=base_model, 
    embedding_dim=EMBEDDING_DIM,
    align_loss_type="cos_align",
    align_model_type="self",
    align_weight=0.5,
    foundation_embedding=None,
)

基础模型初始化完毕，输入特征: 10, 嵌入维度: 64


In [13]:
# 3. 准备模拟数据
x_complete = torch.randn(BATCH_SIZE, SEQ_LEN, FEATURES) # 完整的原始数据
x_masked = x_complete.clone()
# 随机生成一个 mask，遮掉大约 20% 的数据点
mask = torch.rand(x_masked.shape) > 0.8
x_masked[mask] = 0 # 将被遮盖的数据点设为0

In [14]:
# 4. 模拟训练过程
print("🚀 模式: 训练 (Training)")
glocal_model.train() # 将模型设置为训练模式

# 在训练时，需要同时传入 masked 和 complete 数据
training_results = glocal_model(x_masked, x_complete)

print(f"返回结果类型: {type(training_results)}")
print(f"字典的键: {training_results.keys()}")

# 从字典中获取插补结果和对齐损失
imputation = training_results['output']
alignment_loss = training_results['alignment_loss']

print(f"插补结果的形状: {imputation.shape}")
print(f"对齐损失的值: {alignment_loss.item():.4f}")

# 在实际训练中，你会计算一个标准的重建损失
reconstruction_loss = F.mse_loss(imputation[~mask], x_complete[~mask])

# 总损失是对齐损失和重建损失的加权和
total_loss = reconstruction_loss + alignment_loss
print(f"重建损失: {reconstruction_loss.item():.4f}")
print(f"总损失 (用于反向传播): {total_loss.item():.4f}")

🚀 模式: 训练 (Training)
返回结果类型: <class 'dict'>
字典的键: dict_keys(['output', 'alignment_loss'])
插补结果的形状: torch.Size([4, 24, 10])
对齐损失的值: 0.4952
重建损失: 1.0639
总损失 (用于反向传播): 1.5591


In [15]:
# 5. 模拟评估/推理过程
print("🔬 模式: 评估 (Evaluation)")
glocal_model.eval() # 将模型设置为评估模式

# 在评估时，只传入 masked 数据
with torch.no_grad():
    eval_output = glocal_model(x_masked)

print(f"返回结果类型: {type(eval_output)}")
print(f"插补结果的形状: {eval_output.shape}")

🔬 模式: 评估 (Evaluation)
返回结果类型: <class 'torch.Tensor'>
插补结果的形状: torch.Size([4, 24, 10])


In [ ]:
# 6. 演示 __getattr__ 的作用
print("🎁 演示 __getattr__ 功能")
# 尽管我们操作的是 glocal_model，但可以像调用 base_model 的方法一样直接调用
# 这是因为 __getattr__ 会自动将调用转发给内部的 self.base_model
emb_dim = glocal_model.get_embedding_dim()
print(f"通过 Glocal_IB 包装器直接调用基础模型的方法: glocal_model.get_embedding_dim() -> {emb_dim}")
print(f"与直接调用基础模型的结果一致: base_model.get_embedding_dim() -> {base_model.get_embedding_dim()}")

🎁 演示 __getattr__ 功能
通过 Glocal_IB 包装器直接调用基础模型的方法: glocal_model.get_embedding_dim() -> 64
与直接调用基础模型的结果一致: base_model.get_embedding_dim() -> 64
